## Получаем токен и прописываем его в vk_api_utils

In [1]:
from functions.vk_api_utils import UserTokenURL
UserTokenURL()

https://oauth.vk.com/authorize?client_id=6610408&redirect_uri=https://oauth.vk.com/blank.html&scope=wall&response_type=token&v=5.92


In [1]:
import re
import urllib

import pandas as pd
import sqlite3 as sqlite

from functions.vk_api_multiprocess import *
from functions.vk_api import *

%reload_ext autoreload
%autoreload 2

folder = 'data'

## Находим группы в vk

In [2]:
public_groups_ids = MultiGroupsSearch(['шутки', 'анекдоты'], verbose=True, processes=40)
print('Найдено %d групп' % len(public_groups_ids))


Найдено 2000 групп


In [3]:
public_groups_ids

array(['fanta', 'blackumorrus', 'i.love.smile', ..., 'club83227806',
       'club55738082', 'ivlubs'], dtype='<U32')

In [4]:
def check_group_posts(group):
    posts = WallGet(group, count=100)
    
    good_posts = 0
    for post in posts:
        sentence_length = len(re.findall(r'[а-я А-Я]', post['text']))
        if 40 < sentence_length < 400:
            good_posts += 1
    
    if good_posts > 85:
        return True
    
    return False

## Отсеиваем лишние группы 

In [94]:
trusted_groups = []
for group in public_groups_ids:
    if check_group_posts(group):
        trusted_groups.append(group)
len(trusted_groups)

6 Too many requests per second
6 Too many requests per second
6 Too many requests per second
6 Too many requests per second
6 Too many requests per second
6 Too many requests per second
6 Too many requests per second
6 Too many requests per second
6 Too many requests per second
6 Too many requests per second
6 Too many requests per second
6 Too many requests per second
6 Too many requests per second
6 Too many requests per second
6 Too many requests per second
6 Too many requests per second
6 Too many requests per second
6 Too many requests per second
6 Too many requests per second
6 Too many requests per second
6 Too many requests per second
6 Too many requests per second
6 Too many requests per second
6 Too many requests per second
6 Too many requests per second
6 Too many requests per second
6 Too many requests per second
6 Too many requests per second
15 Access denied: this wall available only for community members
6 Too many requests per second
6 Too many requests per second
6 Too

84

## Собираем записи найденных групп

In [193]:
class Database:
    def __init__(self):
        self.conn = sqlite.connect('jokes.db')
        self.cursor = self.conn.cursor()
        sql = """
            CREATE TABLE IF NOT EXISTS jokes
            (jokes text, date int, likes int, comments int, reposts int, views int)
        """
        self.cursor.execute(sql)
        self.conn.commit() 
    
    def add_jokes(self, jokes):
        self.cursor.executemany("INSERT INTO jokes VALUES (?,?,?,?,?,?)",
                    [(joke['text'], joke['date'], 
                      joke['likes']['count'] if 'likes' in joke else None, 
                      joke['comments']['count'] if 'comments' in joke else None, 
                      joke['reposts']['count'] if 'reposts' in joke else None, 
                      joke['views']['count'] if 'views' in joke else None)
                     for joke in jokes])
        self.conn.commit()

In [215]:
db = Database()

In [216]:
def posts_preprocessing(posts):
    good_posts = []
    for post in posts:
        sentence_length = len(re.findall(r'[а-я А-Я]', post['text']))
        if 40 < sentence_length < 300:
            good_posts.append(post)
    return good_posts

In [218]:
for group in trusted_groups:
    print(group)
    offset = 0
    while True:
        try:
            posts = WallGet(group, offset=100*offset)
        except urllib.error.HTTPError:
            break
            
        if not posts:
            break

        db.add_jokes(posts)
        offset += 1

semechkitv
6 Too many requests per second
vishutka 
umorinabest
6 Too many requests per second
kvnclub 
qk.test 
biatlonkvn 
shutky_vk 
smile.like_me 
5xaxa 
aphorisms_vk
6 Too many requests per second
ymor_fm 
only_jokes 
yumora_shutka
6 Too many requests per second
shutkin300 
umor100600 
yumordlyavzrosluh
kazuuussss
6 Too many requests per second
anek_net 
anekdotshort
6 Too many requests per second
anekdotpremium
6 Too many requests per second
world_a_positive
6 Too many requests per second
anekdotlend
6 Too many requests per second
anecdoty2016
6 Too many requests per second
youranektot
6 Too many requests per second
anekdot_e
6 Too many requests per second
anekdotik
6 Too many requests per second
anekdots5good 
1alb_anekdot
6 Too many requests per second
smex_umora
6 Too many requests per second
anekdoty_ru
6 Too many requests per second
anekdotovich 
anekdot79
6 Too many requests per second
1because5
6 Too many requests per second
laughter_inyour_body
6 Too many requests per sec

## Сохраняем в csv

In [24]:
conn = sqlite.connect('jokes.db')
cursor = conn.cursor()
temp = [i for i in cursor.execute("""
SELECT * FROM jokes j
""")]

data = pd.DataFrame(temp)
pd.DataFrame({'text':data[0].values}).to_csv('jokes.csv', index=False)